<a href="https://colab.research.google.com/github/blackteck/BIT-Internship-Group8/blob/main/PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FAKE CELEBRITY IMAGE GENERATION USING GAN


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

# Define the generator
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Define the discriminator
def build_discriminator(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build and compile the discriminator
img_shape = (28, 28, 1)
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

# Build the generator
latent_dim = 100
generator = build_generator(latent_dim)

# The generator takes noise as input and generates images
z = layers.Input(shape=(latent_dim,))
img = generator(z)

# For the combined model, only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
validity = discriminator(img)

# The combined model (stacked generator and discriminator)
combined = tf.keras.Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Load the celebrity face dataset
dataset_path = '/content/data'
celeb_faces = []
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img / 127.5 - 1.0  # Normalize to the range [-1, 1]
    celeb_faces.append(img)

celeb_faces = np.array(celeb_faces)

# Reshape the dataset to have a single channel (grayscale)
celeb_faces = celeb_faces.reshape(celeb_faces.shape + (1,))

# Adversarial ground truths
batch_size = 64  # Adjust the batch size here
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Training parameters
epochs = 10000
save_interval = 1000

# Training the GAN
def save_generated_images(epoch, generator_model):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator_model.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

for epoch in range(epochs):
    # Select a random batch of celebrity faces
    idx = np.random.randint(0, celeb_faces.shape[0], batch_size)
    celeb_imgs = celeb_faces[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(celeb_imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    g_loss = combined.train_on_batch(noise, valid)

    # Print progress and save generated images at specified intervals
    if epoch % save_interval == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
        save_generated_images(epoch, generator)
